# 数据加载

In [1]:
import pandas as pd
import numpy as np
import scipy.io as sio
from scipy import sparse
from sklearn.model_selection import train_test_split
from utils import util
import argparse
import itertools
import time
import torch
from model_multiview import DCPMultiView
from utils.get_mask import get_mask
from utils.util import cal_classify
from utils.logger import get_logger
from utils.datasets import *
from configure.configure_supervised_multiview import get_default_config
import collections

In [2]:
import warnings
warnings.simplefilter("ignore")

In [3]:
data = pd.read_csv(r'C:\Users\psck\Desktop\数据_最终.csv')
train_x1 = data.iloc[:, 0:4].values
train_x2 = data.iloc[:, 4:12].values
train_x3 = data.iloc[:, 12:18].values
train_x4 = data.iloc[:, 18:21].values
train_x5 = data.iloc[:, 21:].values
# print(train_x1)

print("train_x1 前五项：")
print(train_x1[:5])

print("\ntrain_x2 前五项：")
print(train_x2[:5])

print("\ntrain_x3 前五项：")
print(train_x3[:5])

print("\ntrain_x4 前五项：")
print(train_x4[:5])

print("\ntrain_x5 前五项：")
print(train_x5[:5])


train_x1 前五项：
[[71.4  8.   0.   1. ]
 [81.5 18.   0.   0. ]
 [68.6 12.   0.   1. ]
 [86.8 16.   0.   2. ]
 [84.3 18.   3.   1. ]]

train_x2 前五项：
[[  5.    17.    27.     9.    23.     2.   -14.48 -12.09]
 [  4.5   26.67  36.67  10.    24.     1.   -16.62 -15.44]
 [  3.    24.    34.    10.    23.     0.   -15.34 -13.74]
 [  3.5   18.    29.    10.    26.     0.   -11.45 -10.28]
 [  2.5   14.    24.    10.    24.     0.   -10.84 -12.87]]

train_x3 前五项：
[[  28866.    4852.    2177.   13843.   20027. 1361930.]
 [  61993.    4988.    2452.   12337.   13002. 1305270.]
 [  34380.    6330.    2705.   15933.   19961. 1746260.]
 [  46630.    5161.    3385.   17842.   16481. 1549250.]
 [  19619.    4041.    1928.   14607.   16783. 1354830.]]

train_x4 前五项：
[[871.83 306.47  30.18]
 [743.9  442.4   40.09]
 [871.83 306.47  30.18]
 [871.83 306.47  30.18]
 [410.2  362.1   34.9 ]]

train_x5 前五项：
[[ 2.85  3.14  4.98  1.52  5.52  1.27  2.74  2.47  3.35  3.66  0.97  0.82
   3.03  4.99  2.35  5.27  2.48  

# 导入数据

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
# np.set_printoptions(suppress=True, precision=2)
data = pd.read_csv(r'C:\Users\psck\Desktop\数据_最终.csv')
train_x = data.iloc[:, 0:49].values
train_y = data.iloc[:, 49:].values
# print(train_x)
# print(train_y)
X_list=train_x
Y_list=train_y

# print(train_x)
# print(train_y)
print(train_x.shape)

(645, 49)


In [ ]:
x1_train_raw = train_x[:, 0:4]
x2_train_raw = train_x[:, 4:12]
x3_train_raw = train_x[:, 12:18]
x4_train_raw = train_x[:, 18:21]
x5_train_raw = train_x[:, 21:]
label_raw=Y_list

# print(x2_train_raw[:3])
# print(x2_train_raw.shape[1])
# print(x2_train_raw.shape)
# print(x2_train_raw)
# print(x1_train_raw)
len1 = x1_train_raw.shape[1]
len2 = x2_train_raw.shape[1] + x1_train_raw.shape[1]
len3 = x3_train_raw.shape[1] + x2_train_raw.shape[1] + x1_train_raw.shape[1]
len4 = x4_train_raw.shape[1] + x3_train_raw.shape[1] + x2_train_raw.shape[1] + x1_train_raw.shape[1]
len5 =  len4 + x5_train_raw.shape[1]
# data2 = np.concatenate([x1_train_raw, x2_train_raw, x3_train_raw, x4_train_raw, x5_train_raw], axis=1)
# print(len1,len2,len3,len4,len5)

x_train, x_test, labels_train, labels_test = train_test_split(train_x, label_raw, test_size=0.2)  # 测试集的比例为20%
x1_train = x_train[:, 0:len1]
x2_train = x_train[:, len1:len2]
x3_train = x_train[:, len2:len3]
x4_train = x_train[:, len3:len4]
x5_train = x_train[:, len4:len5]

x1_test = x_test[:, :len1]
x2_test = x_test[:, len1:len2]
x3_test = x_test[:, len2:len3]
x4_test = x_test[:, len3:len4]
x5_test = x_test[:, len4:len5]
# 
# GT = torch.from_numpy(labels_train).long()
# print(x5_test)
# print(type(data))
# print(type(train_x))

# print(len1, len2, len3, len4, len5)
# print(x1_train.shape, x2_train.shape, x3_train.shape, x4_train.shape, x5_train.shape)
# print(x1_test.shape, x2_test.shape, x3_test.shape, x4_test.shape, x5_test.shape)
# print(data.head())
# print(x5_test)
# classes = np.unique(Y_list).size
# print(classes)
# print(x1_train)
# print(x4_test)
# print(type(x1_train))
# print(x2_train)
# device = torch.device('cuda' if use_cuda else 'cpu')
# device = torch.device('cuda')
# torch_tensor2 = torch.from_numpy(x2_train)
# print(torch_tensor2)